# Instalación de dependencias para librerías y bases de datos

In [1]:
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.databases #Package for database reading.
!git clone https://github.com/UN-GCPDS/eeg_models.git
FILEID = "1lo0MjWLvsyne2CgTA6VZ2HGY9SKxiwZ7"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O MI_EEG_ClassMeth.zip && rm -rf /tmp/cookies.txt
!unzip MI_EEG_ClassMeth.zip #Package with useful functions for motor imagery classification based in EEG.

  Cloning https://github.com/UN-GCPDS/python-gcpds.databases to /tmp/pip-req-build-y_gva03n
  Running command git clone --filter=blob:none --quiet https://github.com/UN-GCPDS/python-gcpds.databases /tmp/pip-req-build-y_gva03n
  Resolved https://github.com/UN-GCPDS/python-gcpds.databases to commit d174df9958b6638156dcfe03996a6307e631a6a2
  Preparing metadata (setup.py) ... done
  Created wheel for gcpds-databases: filename=gcpds_databases-0.2-py3-none-any.whl size=32972807 sha256=cc1deb5d081a21319f15bb451d2119bf954929e8f001bd88233eeab79f110ce0
  Stored in directory: /tmp/pip-ephem-wheel-cache-194vhxp9/wheels/ae/48/8d/edf617d5fe8f03b17aa26306a04abdfcc605b218d8e6deac83
Successfully built gcpds-databases
Cloning into 'eeg_models'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 115 (delta 65), reused 68 (delta 29), pack-reused 0 (from 0)
Receiving objects: 100% (115/115), 61.53 KiB | 2.

# Importación de librerías

In [2]:
# load databases
from gcpds.databases import GIGA_MI_ME
from gcpds.databases.BCI_Competition_IV import Dataset_2a
import sys
from MI_EEG_ClassMeth.FeatExtraction import TimeFrequencyRpr
sys.path.append('/kaggle/working/eeg_models')

from utils import train
import numpy as np

2025-04-25 00:59:47.631472: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745542787.948249      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745542788.038042      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Creación del modelo

In [3]:
from models import ShallowConvNet

# Definición de las dimensiones de entrada
C = 64  # Número de canales
T = 320  # Número de muestras por segmento

# Parámetros específicos del modelo
model_params = {
    'nb_classes': 2,
    'dropoutRate': 0.5,
    'version': '2018'
}

# Argumentos generales del modelo
model_args = {
    'model_name': 'ShallowConvNet',
    'nb_classes': 2,
    'autoencoder': False
}

# Inicialización del modelo con los parámetros definidos
model = ShallowConvNet(**model_params, Chans=C, Samples=T)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1745542802.386793      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1745542802.387586      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


# Pruebas con GIGA

In [4]:
# Nombre de la base de datos y carga
db_name = 'GIGA_MI_ME'
db_path = '/kaggle/input/giga-science-gcpds/GIGA_MI_ME'
db = GIGA_MI_ME(db_path,)

# Frecuencia de muestreo
fs = db.metadata['sampling_rate']

# Nombres de los canales EEG
eeg_ch_names = [
    'Fp1', 'Fpz', 'Fp2',
    'AF7', 'AF3', 'AFz', 'AF4', 'AF8',
    'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8',
    'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8',
    'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8',
    'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8',
    'P9', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'P10',
    'PO7', 'PO3', 'POz', 'PO4', 'PO8',
    'O1', 'Oz', 'O2',
    'Iz'
]

# Definición de los bancos de frecuencia y ventana temporal
f_bank = np.array([[4., 40.]])        # Banda de frecuencia de interés
vwt = np.array([[2.5, 5]])            # Ventana temporal para la representación TF

# Representación tiempo-frecuencia
tf_repr = TimeFrequencyRpr(sfreq=fs, f_bank=f_bank, vwt=vwt)

# Argumentos para la función de carga de datos
load_args = {
    'db': db,
    'eeg_ch_names': eeg_ch_names,
    'fs': fs,
    'f_bank': f_bank,
    'vwt': vwt,
    'new_fs': 128.0,
    'tf_repr': tf_repr
}

# Selección de sujetos (excluyendo sujetos 29 y 34)
subjects = np.arange(1, db.metadata['subjects'] + 1)
subjects = np.delete(subjects, [28, 33])

In [5]:
# === Imports ===
from tensorflow.keras.callbacks import ReduceLROnPlateau, TerminateOnNaN
from tensorflow.keras.losses import SparseCategoricalCrossentropy, MeanSquaredError
from sklearn.model_selection import StratifiedShuffleSplit

# === Parámetros generales ===
seed = 23
verbose = 0

# === Callbacks de entrenamiento ===
reduce_lr_on_plateau = ReduceLROnPlateau(
    monitor='loss',
    factor=0.1,
    patience=30,
    verbose=verbose,
    mode='min',
    min_delta=0.01,
    min_lr=0
)

terminate_on_nan = TerminateOnNaN()

callbacks = [reduce_lr_on_plateau, terminate_on_nan]

# === Configuración de validación cruzada ===
cv_args = {
    'cv': StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=seed)
}

# === Argumentos de compilación del modelo ===
compile_args = {
    'loss': SparseCategoricalCrossentropy(),  # Alternativa: 'mse' o MeanSquaredError()
    'init_lr': 1e-2
}

# === Argumentos del ajuste (fit) del modelo ===
fit_args = {
    'epochs': 500,
    'verbose': verbose,
    'callbacks': callbacks
}

In [6]:
all_results = []

for sbj in subjects[:]: 
    print('sbj = ', sbj)
    load_args['sbj'] = sbj
    results = train(model, db_name, tf_repr, load_args, cv_args, model_args, compile_args, fit_args, seed)
    all_results.append((sbj, results))

sbj =  1
Resampling from 512.000000 to 128.000000 Hz.


I0000 00:00:1745542813.218261     118 service.cc:148] XLA service 0x7fb0301079a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745542813.219471     118 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1745542813.219492     118 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1745542813.524697     118 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-04-25 01:00:16.656602: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng1{k2=2,k3=0} for conv (f32[160,40,64,308]{3,2,1,0}, u8[0]{0}) custom-call(f32[160,40,1,308]{3,2,1,0}, f32[40,40,64,1]{3,2,1,0}), window={size=64x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 433ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 261ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 269ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 265ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 258ms/step
sbj =  2
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 262ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 263ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 262ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 271ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 266ms/step
sbj =  3
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 257ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 252ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 253ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 259ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 278ms/step
sbj =  4
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 249ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 269ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 299ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 256ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 259ms/step
sbj =  5
Resampling from 512.000000 to 128.000000 Hz.


2025-04-25 01:13:15.222744: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng1{k2=2,k3=0} for conv (f32[159,40,64,308]{3,2,1,0}, u8[0]{0}) custom-call(f32[159,40,1,308]{3,2,1,0}, f32[40,40,64,1]{3,2,1,0}), window={size=64x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]} is taking a while...
2025-04-25 01:13:15.585371: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.362744803s
Trying algorithm eng1{k2=2,k3=0} for conv (f32[159,40,64,308]{3,2,1,0}, u8[0]{0}) custom-call(f32[159,40,1,308]{3,2,1,0}, f32[40,40,64,1]{3,2,1,0}), window={size=64x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"cudnn_conv_backend_config":{"activatio

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 256ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 257ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 269ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 263ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 259ms/step
sbj =  6
Resampling from 512.000000 to 128.000000 Hz.


...

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 260ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 288ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 249ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 253ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 252ms/step
sbj =  47
Resampling from 512.000000 to 128.000000 Hz.


2025-04-25 03:24:23.198907: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng1{k2=2,k3=0} for conv (f32[153,40,64,308]{3,2,1,0}, u8[0]{0}) custom-call(f32[153,40,1,308]{3,2,1,0}, f32[40,40,64,1]{3,2,1,0}), window={size=64x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]} is taking a while...
2025-04-25 03:24:23.497364: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.298557922s
Trying algorithm eng1{k2=2,k3=0} for conv (f32[153,40,64,308]{3,2,1,0}, u8[0]{0}) custom-call(f32[153,40,1,308]{3,2,1,0}, f32[40,40,64,1]{3,2,1,0}), window={size=64x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"cudnn_conv_backend_config":{"activatio

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 285ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 267ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 271ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 272ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 265ms/step
sbj =  48
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 260ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 257ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 249ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 261ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 264ms/step
sbj =  49
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 254ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 278ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 270ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 261ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 257ms/step
sbj =  50
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 265ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 266ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 259ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 269ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 265ms/step
sbj =  51
Resampling from 512.000000 to 128.000000 Hz.


2025-04-25 03:37:50.402966: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng1{k2=2,k3=0} for conv (f32[155,40,64,308]{3,2,1,0}, u8[0]{0}) custom-call(f32[155,40,1,308]{3,2,1,0}, f32[40,40,64,1]{3,2,1,0}), window={size=64x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]} is taking a while...
2025-04-25 03:37:50.715254: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.312387068s
Trying algorithm eng1{k2=2,k3=0} for conv (f32[155,40,64,308]{3,2,1,0}, u8[0]{0}) custom-call(f32[155,40,1,308]{3,2,1,0}, f32[40,40,64,1]{3,2,1,0}), window={size=64x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"cudnn_conv_backend_config":{"activatio

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 260ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 264ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 264ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 268ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 262ms/step
sbj =  52
Resampling from 512.000000 to 128.000000 Hz.
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 271ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 258ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 257ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 265ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 270ms/step


In [7]:
import pickle

with open(f'results_{model_args["model_name"]}.pkl', 'wb') as f:
    pickle.dump(all_results, f)